# Recon3DplusthreeCompetitive

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon3DplusthreeCompetitiveforObesityMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon3DplusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
model=M.copy()

In [4]:
phe_L_c_related=['lyspheile_TOBD','phetyr_TOBD','phetyrgln_TOBD','hisphearg_TOBD','valtrpphe_TOBD','phetyrlys_TOBD',
                    'glyphe_TOBD','lysvalphe_TOBD','thrphearg_TOBD','argleuphe_TOBD','tyralaphe_TOBD','cysaspphe_TOBD',
                    'argphearg_TOBD','glylysphe_TOBD','metphearg_TOBD','mettrpphe_TOBD','prophe_TOBD','pheasnmet_TOBD',
                    'pheasp_TOBD','pheglnphe_TOBD','pheleu_TOBD','pheleuasp_TOBD','pheleuhis_TOBD','tyrphetyr_TOBD',
                    'phelysala_TOBD','phelyspro_TOBD','tyrtrpphe_TOBD','asnpheasp_TOBD','asnphecys_TOBD','phepheasn_TOBD',
                    'asntyrphe_TOBD','serphelys_TOBD','pheproarg_TOBD','valleuphe_TOBD','phesertrp_TOBD','lysglnphe_TOBD',
                    'argalaphe_TOBD','phethrlys_TOBD','trpphe_TOBD','phetrpleu_TOBD','valphearg_TOBD','phephethr_TOBD',
                'phephe_TOBD','glyphe_TOBD','trpglyphe_TOBD']
for reaction in phe_L_c_related:
    model.reactions.get_by_id(reaction).lower_bound=0
    model.reactions.get_by_id(reaction).upper_bound=0

In [5]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [6]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [7]:
for rxn in model.metabolites.phe_L_c.reactions:
    model.reactions.get_by_id(rxn.id).upper_bound=0
    model.reactions.PheTR_TOBD.upper_bound=1000
    model.reactions.biomass_reaction.upper_bound=1000
    model.reactions.r0399.upper_bound=1000
    model.reactions.PHETHPTOX2.upper_bound=1000

In [8]:
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# Phe secretion in the blood is blocked, otherwise there is no competition.
model.reactions.EX_phe_L_BD.upper_bound=0
# No phenylpyruvate production in the brain and liver
model.reactions.PHETA1.upper_bound=0
model.reactions.PHETA1_INB.upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
### No Tyr hydroxylase competition
model.reactions.Phe_TH_INB.upper_bound=0
model.reactions.PhePde_INB.upper_bound=0
model.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti1_To1.upper_bound=0.195
model.solver='glpk'
sol = model.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2517557

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [9]:
### Essential need for model
model.reactions.biomass_reaction_INB.lower_bound=0.1
model.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.0567
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095

In [10]:
model.objective='EX_tag_hs_BD'
model.solver='glpk'
sol = model.optimize()
sol.objective_value

0.036507422342019015

In [11]:
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [12]:
# To simulate the case for PKU patient (low Phe diet and high glucose)
model1=model.copy()
# Kncoking gene '5053.1' to simulate the PKU patients
model1.genes.get_by_id('5053.1').knock_out()
# No phenylpyruvate production in the brain and liver
model1.reactions.PHETA1.upper_bound=0.1
model1.reactions.PHETA1_INB.upper_bound=0.1
model1.reactions.EX_phe_L_e.upper_bound=-0.05
model1.reactions.EX_phe_L_e.lower_bound=-0.05
## The Phe-restricted diet contains more carbohydrate content, We assumed more glucose.
model1.reactions.EX_glc_D_e.lower_bound=-2
sol = model1.optimize()
sol.objective_value

0.10012162264150908

In [13]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [14]:
## Normal person uptake more glucose
model2=model.copy()
model2.reactions.EX_glc_D_e.lower_bound=-2
sol = model2.optimize()
sol.objective_value

0.11317708391734049

In [15]:
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [16]:
## PKU patient uptake same glucose as normal person
model3=model.copy()
# Kncoking gene '5053.1' to simulate the PKU patients
model3.genes.get_by_id('5053.1').knock_out()
# No phenylpyruvate production in the brain and liver
model3.reactions.PHETA1.upper_bound=0.1
model3.reactions.PHETA1_INB.upper_bound=0.1
model3.reactions.EX_phe_L_e.upper_bound=-0.05
model3.reactions.EX_phe_L_e.lower_bound=-0.05
## The Phe-restricted diet contains more carbohydrate content, We assumed more glucose.
model3.reactions.EX_glc_D_e.lower_bound=-1
sol = model3.optimize()
sol.objective_value

0.023451961066187767

In [17]:
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [20]:
model.reactions.EX_2hyoxplac_e

Reaction identifier,EX_2hyoxplac_e
Name,2-Hydroxyphenylacetate
Memory address,0x12952ff70
Stoichiometry,2hyoxplac_e --> 2-Hydroxyphenylacetate -->
GPR,
Lower bound,0.0
Upper bound,0.0
